# Data analysis with pandas - Hockey fights

David Singer, the gentleman who runs [hockeyfights dot com](http://www.hockeyfights.com/), was kind enough to provide us with a cut of the data powering his website for us to use in training sessions. Thanks, David!

This data lives here: `../../data/hockey-fights.xlsx`. Every row in the data is one fight.

First, import pandas, then use the `read_excel()` method to load the data into a dataframe.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('../../data/hockey-fights.xlsx', sheet_name='fights')

In [ ]:
df.head()

### Check out the data

In [ ]:
df.info()

In [ ]:
df.date.min()

In [ ]:
df.date.max()

In [ ]:
df.away_team_name.unique()

In [ ]:
df.home_team_name.unique()

In [ ]:
# etc ...

### Come up with a list of questions

- Which player was involved in the most fights?
- Average number of fights per game?
- What was the longest fight?

... what else?

### Q: Which player was involved in the most fights?

This one will be a little tricky because of how the data is structured -- a player could be fighting either as the home or away player, so there's not an obvious column to group or pivot on. There are a couple of strategies we could use to answer this question. Here's what we're going to do:

- Use the [`concat()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html) method to stack the column values in each player ID column into one Series (we're using player ID instead of name to avoid the "John Smith" problem (or I guess "Graham MacKenzie")
- Use `value_counts()` to get a count
- Grab the player ID with the most fights by getting the first ([0]) element in the `index` list for the Series returned by `value_counts()`
- Go back to the original data frame and filter for that ID, then use [`iloc`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.iloc.html) to get a single fight record with the player's name, team, etc.

In [ ]:
all_player_ids = pd.concat([df.home_player_id, df.away_player_id])

top = all_player_ids.value_counts().index[0]

player_record = df[df.away_player_id == top].iloc[0]
print(player_record.away_player_name, player_record.away_team_name)

### Q: Average number of fights per game?

This one will be pretty easy. We need two numbers: The total number of fights -- which is the same as asking how many records are in our data frame -- and the total number of games, which will just involve counting the unique number of games in our data.

To get the number of records in our data frame, I shall use the `shape` attribute, which returns a [tuple](https://www.tutorialspoint.com/python/python_tuples.htm) with two things: the number of rows (the first thing) and the number of columns (the second thing). You can access items in a tuple just like you'd access items in a list: With square brackets `[]` and the index number of the thing you're trying to get.

To get the number of unique games, we're going to use the [`nunique()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nunique.html) method to get the number of unique game IDs.

(Note: I did not know about the `nunique()` method until I Googled "pandas count unique values series," so.)

In [ ]:
num_fights = df.shape[0]
num_games = df.game_id.nunique()
avg_fights_per_game = num_fights / num_games

print(avg_fights_per_game)

### Q: What was the longest fight?

We have fight duration as a mixture of minutes and seconds, so we first need to convert to seconds ((minutes * 60) + seconds). We'll create a new column, `fight_duration`, for this. Then it's just a matter of sorting top to bottom by that new column and using [`.iloc[0]`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.iloc.html) again to grab the first record.

In [ ]:
df['fight_duration'] = (df.fight_minutes * 60) + df.fight_seconds
df.sort_values('fight_duration', ascending=False).iloc[0]